# Instacart market basket analysis

## Data Mining and analysis CBD-3334_1

* Andrea Franco - C0931897

# Libraries

In [3]:
import pandas as pd
import sklearn
import numpy as np


# Exploratory Data Analysis

In [4]:
aisles_df = pd.read_csv("market_basket_analysis/datasets/aisles.csv")
order_prods_prior_df = pd.read_csv("market_basket_analysis/datasets/order_products__prior.csv")
order_prods_train_df = pd.read_csv("market_basket_analysis/datasets/order_products__train.csv")
orders_df = pd.read_csv("market_basket_analysis/datasets/orders.csv")
products_df = pd.read_csv("market_basket_analysis/datasets/products.csv")

In [5]:
print("aisles shape: ", aisles_df.shape)
print("order_prods_prior shape: ", order_prods_prior_df.shape)
print("order_prods_train shape: ", order_prods_train_df.shape)
print("orders shape: ", orders_df.shape)
print("products shape: ", products_df.shape)

aisles shape:  (134, 2)
order_prods_prior shape:  (32434489, 4)
order_prods_train shape:  (1384617, 4)
orders shape:  (3421083, 7)
products shape:  (49688, 4)


In [6]:
aisles_df.head()

,aisle_id,aisle
0,1,prepared soups salads
1,2,specialty cheeses
2,3,energy granola bars
3,4,instant foods
4,5,marinades meat preparation


Reduce the number of aisles 

In [7]:
aisles_df['aisle'].unique()

array(['prepared soups salads', 'specialty cheeses',
       'energy granola bars', 'instant foods',
       'marinades meat preparation', 'other', 'packaged meat',
       'bakery desserts', 'pasta sauce', 'kitchen supplies',
       'cold flu allergy', 'fresh pasta', 'prepared meals',
       'tofu meat alternatives', 'packaged seafood', 'fresh herbs',
       'baking ingredients', 'bulk dried fruits vegetables',
       'oils vinegars', 'oral hygiene', 'packaged cheese', 'hair care',
       'popcorn jerky', 'fresh fruits', 'soap', 'coffee', 'beers coolers',
       'red wines', 'honeys syrups nectars', 'latino foods',
       'refrigerated', 'packaged produce', 'kosher foods',
       'frozen meat seafood', 'poultry counter', 'butter',
       'ice cream ice', 'frozen meals', 'seafood counter',
       'dog food care', 'cat food care', 'frozen vegan vegetarian',
       'buns rolls', 'eye ear care', 'candy chocolate', 'mint gum',
       'vitamins supplements', 'breakfast bars pastries',
       '

Map into 5 big categories

In [8]:
def categorize_aisle(aisle):
    fresh_foods = ['fresh fruits', 'fresh vegetables', 'fresh herbs', 'poultry counter', 'seafood counter', 'meat counter', 'fresh dips tapenades']
    packaged_foods = ['energy granola bars', 'instant foods', 'bakery desserts', 'pasta sauce', 'baking ingredients', 'bulk dried fruits vegetables', 'popcorn jerky', 'candy chocolate', 'cookies cakes', 'crackers', 'chips pretzels', 'cereal', 'dry pasta', 'grains rice dried goods', 'packaged vegetables fruits', 'trail mix snack mix', 'soup broth bouillon', 'canned meals beans', 'canned jarred vegetables', 'canned meat seafood', 'canned fruit applesauce', 'spices seasonings', 'condiments', 'granola', 'preserved dips spreads', 'salad dressing toppings']
    beverages = ['coffee', 'tea', 'juice nectars', 'soft drinks', 'water seltzer sparkling water', 'energy sports drinks', 'protein meal replacements', 'beers coolers', 'red wines', 'white wines', 'spirits', 'specialty wines champagnes', 'cocoa drink mixes', 'frozen juice']
    household_care = ['kitchen supplies', 'oral hygiene', 'soap', 'paper goods', 'shave needs', 'diapers wipes', 'trash bags liners', 'eye ear care', 'vitamins supplements', 'facial care', 'dish detergents', 'laundry', 'deodorants', 'air fresheners candles', 'baby bath body care', 'skin care', 'plates bowls cups flatware', 'cleaning products', 'first aid', 'feminine care', 'body lotions soap', 'muscles joints pain relief', 'beauty']
    frozen_refrigerated = ['frozen meat seafood', 'frozen meals', 'frozen vegan vegetarian', 'frozen breads doughs', 'frozen breakfast', 'ice cream ice', 'ice cream toppings', 'refrigerated', 'milk', 'yogurt', 'butter', 'cream', 'soy lactosefree', 'refrigerated pudding desserts', 'frozen produce', 'frozen pizza', 'frozen appetizers sides', 'frozen dessert']
    
    if aisle in fresh_foods:
        return 'Fresh Foods'
    elif aisle in packaged_foods:
        return 'Packaged Foods'
    elif aisle in beverages:
        return 'Beverages'
    elif aisle in household_care:
        return 'Household & Personal Care'
    elif aisle in frozen_refrigerated:
        return 'Frozen & Refrigerated Items'
    else:
        return 'Other'

# Apply the function to your dataframe
aisles_df['aisle_category'] = aisles_df['aisle'].apply(categorize_aisle)


In [9]:
aisles_df.head()

,aisle_id,aisle,aisle_category
0,1,prepared soups salads,Other
1,2,specialty cheeses,Other
2,3,energy granola bars,Packaged Foods
3,4,instant foods,Packaged Foods
4,5,marinades meat preparation,Other


In [10]:
aisles_df['aisle_category'].value_counts()

aisle_category
Other                          46
Packaged Foods                 26
Household & Personal Care      23
Frozen & Refrigerated Items    18
Beverages                      14
Fresh Foods                     7
Name: count, dtype: int64

In [11]:
order_prods_prior_df.head()

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


In [12]:
orders_df.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


In [13]:
products_df.head()

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


In [14]:
departments_df = pd.read_csv("market_basket_analysis/datasets/departments.csv")
departments_df.head()

,department_id,department
0,1,frozen
1,2,other
2,3,bakery
3,4,produce
4,5,alcohol


# 2. Merge datasets

### Order-Product Details: 
First, we merge the order_products data with orders to attach order-specific information to each product ordered.

In [15]:
order_products_merged = order_prods_prior_df.merge(orders_df, on='order_id', how='left')

In [16]:
order_products_merged.head()

,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2,33120,1,1,202279,prior,3,5,9,8.0
1,2,28985,2,1,202279,prior,3,5,9,8.0
2,2,9327,3,0,202279,prior,3,5,9,8.0
3,2,45918,4,1,202279,prior,3,5,9,8.0
4,2,30035,5,0,202279,prior,3,5,9,8.0


### Add Product Information: 
Merge the order_products_merged with products to attach product-specific details.

In [17]:
order_products_merged = order_products_merged.merge(products_df, on='product_id', how='left')


In [18]:
order_products_merged.head()

,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_name,aisle_id,department_id
0,2,33120,1,1,202279,prior,3,5,9,8.0,Organic Egg Whites,86,16
1,2,28985,2,1,202279,prior,3,5,9,8.0,Michigan Organic Kale,83,4
2,2,9327,3,0,202279,prior,3,5,9,8.0,Garlic Powder,104,13
3,2,45918,4,1,202279,prior,3,5,9,8.0,Coconut Butter,19,13
4,2,30035,5,0,202279,prior,3,5,9,8.0,Natural Sweetener,17,13


### Add Aisle and Department Information: 
Next, merge order_products_merged with aisles and departments to include aisle and department names.

In [19]:
order_products_merged = order_products_merged.merge(aisles_df, on='aisle_id', how='left')
order_products_merged = order_products_merged.merge(departments_df, on='department_id', how='left')


In [20]:
order_products_merged.head()

,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_name,aisle_id,department_id,aisle,aisle_category,department
0,2,33120,1,1,202279,prior,3,5,9,8.0,Organic Egg Whites,86,16,eggs,Other,dairy eggs
1,2,28985,2,1,202279,prior,3,5,9,8.0,Michigan Organic Kale,83,4,fresh vegetables,Fresh Foods,produce
2,2,9327,3,0,202279,prior,3,5,9,8.0,Garlic Powder,104,13,spices seasonings,Packaged Foods,pantry
3,2,45918,4,1,202279,prior,3,5,9,8.0,Coconut Butter,19,13,oils vinegars,Other,pantry
4,2,30035,5,0,202279,prior,3,5,9,8.0,Natural Sweetener,17,13,baking ingredients,Packaged Foods,pantry


In [21]:
order_products_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32434489 entries, 0 to 32434488
Data columns (total 16 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   order_id                int64  
 1   product_id              int64  
 2   add_to_cart_order       int64  
 3   reordered               int64  
 4   user_id                 int64  
 5   eval_set                object 
 6   order_number            int64  
 7   order_dow               int64  
 8   order_hour_of_day       int64  
 9   days_since_prior_order  float64
 10  product_name            object 
 11  aisle_id                int64  
 12  department_id           int64  
 13  aisle                   object 
 14  aisle_category          object 
 15  department              object 
dtypes: float64(1), int64(10), object(5)
memory usage: 3.9+ GB


In [22]:
order_products_merged.isnull().sum()

order_id                        0
product_id                      0
add_to_cart_order               0
reordered                       0
user_id                         0
eval_set                        0
order_number                    0
order_dow                       0
order_hour_of_day               0
days_since_prior_order    2078068
product_name                    0
aisle_id                        0
department_id                   0
aisle                           0
aisle_category                  0
department                      0
dtype: int64

# Data engineering

## Average Days Between Purchases:

We can calculate the average days_since_prior_order for each user to capture their shopping frequency using the user_id


In [21]:
avg_days_between = order_products_merged.groupby('user_id')['days_since_prior_order'].mean().reset_index()
avg_days_between.columns = ['user_id', 'avg_days_between_purchases']


In [22]:
avg_days_between.head()

,user_id,avg_days_between_purchases
0,1,20.259259
1,2,15.967033
2,3,11.487179
3,4,15.357143
4,5,14.500000


In [23]:
avg_days_between.isnull().sum()

user_id                       0
avg_days_between_purchases    0
dtype: int64

## Total Number of Orders:
Count the number of unique order_id values for each user_id.


In [24]:
total_orders = order_products_merged.groupby('user_id')['order_id'].nunique().reset_index()
total_orders.columns = ['user_id', 'total_orders']


In [25]:
total_orders.head()

,user_id,total_orders
0,1,10
1,2,14
2,3,12
3,4,5
4,5,4


## Average Number of Items per Order:
Average count of product_id per order_id for each user.


In [26]:
items_per_order = order_products_merged.groupby(['user_id', 'order_id'])['product_id'].count().reset_index()
avg_items_per_order = items_per_order.groupby('user_id')['product_id'].mean().reset_index()
avg_items_per_order.columns = ['user_id', 'avg_items_per_order']


In [27]:
avg_items_per_order.head()

,user_id,avg_items_per_order
0,1,5.900000
1,2,13.928571
2,3,7.333333
3,4,3.600000
4,5,9.250000


## Most Frequent Day of the Week for Orders:
Most common order_dow (day of the week) for each user.



In [28]:
most_freq_dow = order_products_merged.groupby('user_id')['order_dow'].agg(lambda x: x.mode()[0]).reset_index()
most_freq_dow.columns = ['user_id', 'most_freq_day']


In [29]:
most_freq_dow.head()

,user_id,most_freq_day
0,1,4
1,2,2
2,3,0
3,4,4
4,5,3


## Most Frequent Hour of the Day for Orders:
Mode of order_hour_of_day for each user.

In [30]:
most_freq_hour = order_products_merged.groupby('user_id')['order_hour_of_day'].agg(lambda x: x.mode()[0]).reset_index()
most_freq_hour.columns = ['user_id', 'most_freq_hour']

In [31]:
most_freq_hour.head()

,user_id,most_freq_hour
0,1,7
1,2,9
2,3,16
3,4,15
4,5,18


## Average Time Between Orders for Specific Products:
For each user-product pair, the average days_since_prior_order.


In [32]:
avg_days_product = order_products_merged.groupby(['user_id', 'product_id'])['days_since_prior_order'].mean().reset_index()
avg_days_product.columns = ['user_id', 'product_id', 'avg_days_product_purchase']


In [33]:
avg_days_product.head()

,user_id,product_id,avg_days_product_purchase
0,1,196,19.555556
1,1,10258,19.555556
2,1,10326,28.000000
3,1,12427,19.555556
4,1,13032,21.666667


## Percentage of Orders with a Specific Product:

How often each product appears in orders as a proportion of the user’s total orders.



In [34]:
product_orders = order_products_merged.groupby(['user_id', 'product_id'])['order_id'].nunique().reset_index()
total_orders = order_products_merged.groupby('user_id')['order_id'].nunique().reset_index()
merged = product_orders.merge(total_orders, on='user_id', suffixes=('_product', '_total'))
merged['product_order_ratio'] = merged['order_id_product'] / merged['order_id_total']


In [35]:
merged.head()

,user_id,product_id,order_id_product,order_id_total,product_order_ratio
0,1,196,10,10,1.0
1,1,10258,9,10,0.9
2,1,10326,1,10,0.1
3,1,12427,10,10,1.0
4,1,13032,3,10,0.3


In [36]:
product_order_ratio = merged.copy()

## Recency of Last Purchase for Each Product:

 Identify the last order number for each product and calculate the difference from the user’s most recent order.


In [37]:
last_purchase = order_products_merged.groupby(['user_id', 'product_id'])['order_number'].max().reset_index()
recent_order = order_products_merged.groupby('user_id')['order_number'].max().reset_index()
merged = last_purchase.merge(recent_order, on='user_id', suffixes=('_product', '_recent'))
merged['recency_last_purchase'] = merged['order_number_recent'] - merged['order_number_product']


In [38]:
recency_last_purchase = merged.copy()

In [39]:
recency_last_purchase.head()

,user_id,product_id,order_number_product,order_number_recent,recency_last_purchase
0,1,196,10,10,0
1,1,10258,10,10,0
2,1,10326,5,10,5
3,1,12427,10,10,0
4,1,13032,10,10,0


## Reorder Ratio for Each Product:

Calculate the reorder rate as the ratio of times a product was reordered to the total times it was ordered.

Source Features: user_id, product_id, reordered


In [40]:
product_reorders = order_products_merged.groupby(['user_id', 'product_id'])['reordered'].sum().reset_index()
product_orders = order_products_merged.groupby(['user_id', 'product_id'])['order_id'].count().reset_index()
merged = product_reorders.merge(product_orders, on=['user_id', 'product_id'])
merged['reorder_ratio'] = merged['reordered'] / merged['order_id']

In [41]:
reorder_ratio = merged.copy()

In [42]:
reorder_ratio.head()

,user_id,product_id,reordered,order_id,reorder_ratio
0,1,196,9,10,0.900000
1,1,10258,8,9,0.888889
2,1,10326,0,1,0.000000
3,1,12427,9,10,0.900000
4,1,13032,2,3,0.666667


# Product Popularity in Each Aisle:
Calculate the total orders containing products in each aisle.

Source Features: product_id, aisle_id, order_id


In [43]:
aisle_popularity = order_products_merged.groupby('aisle_id')['order_id'].nunique().reset_index()
aisle_popularity.columns = ['aisle_id', 'aisle_popularity']

In [44]:
aisle_popularity.head()

,aisle_id,aisle_popularity
0,1,63115
1,2,77171
2,3,278151
3,4,165541
4,5,58390


In [45]:
import pandas as pd

# Merge order details to get days_since_prior_order for each user-product order
merged_orders = order_prods_prior_df.merge(orders_df[['order_id', 'user_id', 'order_number', 'days_since_prior_order']], 
                                           on='order_id', how='left')

# Sort by user, product, and order number to get chronological order for each product per user
merged_orders = merged_orders.sort_values(by=['user_id', 'product_id', 'order_number'])

# Calculate days to the next purchase for each user-product combination
merged_orders['days_until_next_order'] = merged_orders.groupby(['user_id', 'product_id'])['days_since_prior_order'].shift(-1)

# Reset index for cleaner merging if needed
merged_orders.reset_index(drop=True, inplace=True)

# At this point, 'days_until_next_order' represents the next purchase time
# If a row has NaN for 'days_until_next_order', it indicates the last purchase for that user-product

# Keep only the relevant columns for the modeling DataFrame
next_purchase_time = merged_orders[['user_id', 'product_id', 'order_id', 'days_until_next_order']].copy()

# Rename 'days_until_next_order' to 'next_purchase_time' for clarity
next_purchase_time.rename(columns={'days_until_next_order': 'next_purchase_time'}, inplace=True)


In [46]:
next_purchase_time.head()

,user_id,product_id,order_id,next_purchase_time
0,1,196,2539329,15.0
1,1,196,2398795,21.0
2,1,196,473747,29.0
3,1,196,2254736,28.0
4,1,196,431534,19.0


In [47]:
next_purchase_time.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32434489 entries, 0 to 32434488
Data columns (total 4 columns):
 #   Column              Dtype  
---  ------              -----  
 0   user_id             int64  
 1   product_id          int64  
 2   order_id            int64  
 3   next_purchase_time  float64
dtypes: float64(1), int64(3)
memory usage: 989.8 MB


## Combining Features for Modeling


### Merge User-Level Features: 

In [48]:
# Assume user_features contains user-level features like 'avg_days_between_purchases', 'total_orders', etc.
user_features = avg_days_between.merge(total_orders, on='user_id', how='left')
user_features = user_features.merge(avg_items_per_order, on='user_id', how='left')
user_features = user_features.merge(most_freq_dow, on='user_id', how='left')
user_features = user_features.merge(most_freq_hour, on='user_id', how='left')


### Merge Product-Level Features: 

In [49]:
# Assume product_features contains product-level features like 'avg_days_product_purchase', 'product_order_ratio', etc.
product_features = avg_days_product.merge(product_order_ratio, on=['user_id', 'product_id'], how='left')

In [50]:
product_features.head()

,user_id,product_id,avg_days_product_purchase,order_id_product,order_id_total,product_order_ratio
0,1,196,19.555556,10,10,1.0
1,1,10258,19.555556,9,10,0.9
2,1,10326,28.000000,1,10,0.1
3,1,12427,19.555556,10,10,1.0
4,1,13032,21.666667,3,10,0.3


### Merge User-Product Interaction Features: 

In [51]:
# Assume user_product_features contains user-product interaction features like 'recency_last_purchase', 'reorder_ratio', etc.
user_product_features = recency_last_purchase.merge(reorder_ratio, on=['user_id', 'product_id'], how='left')


In [52]:
user_product_features.head()

,user_id,product_id,order_number_product,order_number_recent,recency_last_purchase,reordered,order_id,reorder_ratio
0,1,196,10,10,0,9,10,0.900000
1,1,10258,10,10,0,8,9,0.888889
2,1,10326,5,10,5,0,1,0.000000
3,1,12427,10,10,0,9,10,0.900000
4,1,13032,10,10,0,2,3,0.666667


In [ ]:
user_product_features.to_csv('user_product_features.csv', index=False)
user_features.to_csv('user_features.csv', index=False)
product_features.to_csv('product_features.csv', index=False)


# Predicting Product Demand Frequency

In [24]:
# Calculate average days since prior order for each product
avg_demand_interval = order_products_merged.groupby('product_id')['days_since_prior_order'].mean().reset_index()
avg_demand_interval.columns = ['product_id', 'average_demand_interval']
avg_demand_interval.head()


,product_id,average_demand_interval
0,1,10.432725
1,2,10.482759
2,3,10.565385
3,4,14.686207
4,5,12.428571


In [25]:
product_order_counts = order_products_merged.groupby('product_id').size().reset_index(name='total_orders')


In [26]:
avg_cart_position = order_products_merged.groupby('product_id')['add_to_cart_order'].mean().reset_index()
avg_cart_position.columns = ['product_id', 'average_cart_position']


In [27]:
product_reorder_counts = order_products_merged.groupby('product_id').agg(
    total_orders=('order_id', 'count'),
    reorders=('reordered', 'sum')
).reset_index()
product_reorder_counts['reorder_probability'] = product_reorder_counts['reorders'] / product_reorder_counts['total_orders']


In [28]:
unique_user_counts = order_products_merged.groupby('product_id')['user_id'].nunique().reset_index()
unique_user_counts.columns = ['product_id', 'unique_users']


In [29]:
recent_purchase = order_products_merged.groupby('product_id')['days_since_prior_order'].max().reset_index()
recent_purchase.columns = ['product_id', 'recency_last_purchase']


In [30]:
user_product_intervals = order_products_merged.groupby(['product_id', 'user_id'])['days_since_prior_order'].mean().reset_index()
avg_user_product_interval = user_product_intervals.groupby('product_id')['days_since_prior_order'].mean().reset_index()
avg_user_product_interval.columns = ['product_id', 'avg_user_order_interval']


In [31]:
# Start with the target variable DataFrame
modeling_df = avg_demand_interval

# Merge each feature into the modeling DataFrame
modeling_df = modeling_df.merge(product_order_counts, on='product_id', how='left')
modeling_df = modeling_df.merge(avg_cart_position, on='product_id', how='left')
modeling_df = modeling_df.merge(product_reorder_counts[['product_id', 'reorder_probability']], on='product_id', how='left')
modeling_df = modeling_df.merge(unique_user_counts, on='product_id', how='left')
modeling_df = modeling_df.merge(recent_purchase, on='product_id', how='left')
modeling_df = modeling_df.merge(avg_user_product_interval, on='product_id', how='left')

# Example of encoding categorical features if needed
# modeling_df = pd.get_dummies(modeling_df, columns=['aisle', 'department'])


In [32]:
modeling_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49677 entries, 0 to 49676
Data columns (total 8 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   product_id               49677 non-null  int64  
 1   average_demand_interval  49646 non-null  float64
 2   total_orders             49677 non-null  int64  
 3   average_cart_position    49677 non-null  float64
 4   reorder_probability      49677 non-null  float64
 5   unique_users             49677 non-null  int64  
 6   recency_last_purchase    49646 non-null  float64
 7   avg_user_order_interval  49646 non-null  float64
dtypes: float64(5), int64(3)
memory usage: 3.0 MB


In [33]:
products_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49688 entries, 0 to 49687
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   product_id     49688 non-null  int64 
 1   product_name   49688 non-null  object
 2   aisle_id       49688 non-null  int64 
 3   department_id  49688 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 1.5+ MB


In [35]:
modeling_df.isnull().sum()

product_id                  0
average_demand_interval    31
total_orders                0
average_cart_position       0
reorder_probability         0
unique_users                0
recency_last_purchase      31
avg_user_order_interval    31
dtype: int64

In [ ]:
# Step 6: Handle missing values
modeling_df.fillna(0, inplace=True)
modeling_df.dropna(subset=['average_demand_interval'], inplace=True)

# Verify final DataFrame
modeling_df.head()

   product_id  average_demand_interval  total_orders  average_cart_position  \
0           1                10.432725          1852               5.801836   
1           2                10.482759            90               9.888889   
2           3                10.565385           277               6.415162   
3           4                14.686207           329               9.507599   
4           5                12.428571            15               6.466667   

   reorder_probability  unique_users  recency_last_purchase  \
0             0.613391           716                   30.0   
1             0.133333            78                   30.0   
2             0.732852            74                   30.0   
3             0.446809           182                   30.0   
4             0.600000             6                   30.0   

   avg_user_order_interval  
0                13.192109  
1                10.891111  
2                12.801430  
3                14.832102  
4

In [38]:
modeling_df.isnull().sum()

product_id                 0
average_demand_interval    0
total_orders               0
average_cart_position      0
reorder_probability        0
unique_users               0
recency_last_purchase      0
avg_user_order_interval    0
dtype: int64

In [39]:
modeling_df.describe()

,product_id,average_demand_interval,total_orders,average_cart_position,reorder_probability,unique_users,recency_last_purchase,avg_user_order_interval
count,49677.000000,49677.000000,49677.000000,49677.000000,49677.000000,49677.000000,49677.000000,49677.000000
mean,24843.417356,11.557727,652.907563,9.097568,0.366461,267.889627,28.604827,12.308895
std,14343.034804,2.775859,4792.114416,2.551267,0.208103,1308.788623,4.489879,2.802454
min,1.000000,0.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000
25%,12423.000000,10.014505,17.000000,7.625850,0.208075,11.000000,30.000000,10.853783
50%,24842.000000,11.387000,60.000000,9.057269,0.376623,35.000000,30.000000,12.333333
75%,37264.000000,12.916667,260.000000,10.356401,0.529307,137.000000,30.000000,13.750000
max,49688.000000,30.000000,472565.000000,53.000000,0.941176,73956.000000,30.000000,30.000000


In [40]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler

scaler = StandardScaler()  # Or choose another scaler as needed
columns_to_scale = ['average_demand_interval', 'total_orders', 'average_cart_position', 'reorder_probability', 
                    'unique_users', 'recency_last_purchase', 'avg_user_order_interval']
df_scaled = modeling_df.copy()
df_scaled[columns_to_scale] = scaler.fit_transform(df_scaled[columns_to_scale])


In [41]:
df_scaled.describe()

,product_id,average_demand_interval,total_orders,average_cart_position,reorder_probability,unique_users,recency_last_purchase,avg_user_order_interval
count,49677.000000,4.967700e+04,4.967700e+04,4.967700e+04,4.967700e+04,4.967700e+04,4.967700e+04,4.967700e+04
mean,24843.417356,5.725592e-16,8.796501e-18,7.895396e-17,-5.807121e-17,-1.301596e-17,3.520746e-16,-3.803235e-16
std,14343.034804,1.000010e+00,1.000010e+00,1.000010e+00,1.000010e+00,1.000010e+00,1.000010e+00,1.000010e+00
min,1.000000,-4.163700e+00,-1.360389e-01,-3.173972e+00,-1.760974e+00,-2.039232e-01,-6.371021e+00,-4.392229e+00
25%,12423.000000,-5.559495e-01,-1.327001e-01,-5.768634e-01,-7.611027e-01,-1.962824e-01,3.107404e-01,-5.192330e-01
50%,24842.000000,-6.150487e-02,-1.237269e-01,-1.579598e-02,4.883371e-02,-1.779447e-01,3.107404e-01,8.720267e-03
75%,37264.000000,4.895616e-01,-8.199126e-02,4.934199e-01,7.825305e-01,-1.000092e-01,3.107404e-01,5.142347e-01
max,49688.000000,6.643875e+00,9.847779e+01,1.720826e+01,2.761710e+00,5.630310e+01,3.107404e-01,6.312782e+00


In [42]:
df_scaled.to_csv('modeling_df.csv', index=False)